In [1]:
import os
import glob
import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from scipy import stats
import pandas as pd


from bokeh.layouts import row
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#Functions
def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians

def combine_csv_files(root_path, data_path, save_directory, saved_file, Combine_Data):
    os.chdir(data_path)
    extension = 'csv'
    list_csv = []
    for root, dirs, files in os.walk(data_path, topdown=False):
        for name in files:
            if extension in name:
                list_csv.append(os.path.join(root, name))
    #print(list_csv)
    os.chdir(root_path)
    new_dir = save_directory
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
    os.chdir(new_dir)
    combined_csv = pd.concat([pd.read_csv(f) for f in list_csv ], sort=False)
    raw_names=list(combined_csv.Label.unique())
    #print(raw_names)
    if not Combine_Data:
        combined_csv['Strain'] = 'nan'
        combined_csv['Growth'] = 'nan'
        for file in raw_names:
            #print(file)
            split_name=file.rsplit('_event',100)[0].rsplit('_',100)
            if 'Agarose' in split_name or 'sol' in split_name or 'Agar' in split_name:
                growth='Solid'
                if 'Agarose' in split_name:
                    growthInd = split_name.index('Agarose')
                elif 'sol' in split_name:
                    growthInd = split_name.index('sol')
                elif 'Agar' in split_name:
                    growthInd = split_name.index('Agar')
            else:
                growth='Liquid'
                if 'liq' in split_name:
                    growthInd = split_name.index('liq')
                else:
                    growthInd = len(split_name)-3
            separator = '_'
            strain =  separator.join(split_name[0:growthInd])
            combined_csv.Strain.loc[(combined_csv['Label']==file)]=strain
            combined_csv.Growth.loc[(combined_csv['Label']==file)]=growth
    combined_csv.to_csv( saved_file, index=False, encoding='utf-8-sig')
    return combined_csv

In [3]:
#set root folder
root_path = "C:/Users/tala/Desktop/git/PhD_codes/Mechanosensation/Python_code/Pole_analysis/"
#set data folders
data_path_old=[None]*4
data_path_old[0]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/mNG_pilH/'
data_path_old[1]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/pilB_mNG/'
data_path_old[2]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/January/pilT/'
data_path_old[3]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/January/pilU/'
data_path=[None]*5
data_path[0]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/Mars/11/pilH'
data_path[1]='R:/Lorenzo/iSCAT/iSCAT Data/Lorenzo/2020/avril/29/mNG_pilG_fliC-_sol_Gasket_0'
data_path[2]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/Mars/11/pilB'
data_path[3]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/Mars/11/pilT'
data_path[4]='F:/Sauvegardes PhD/Covid-19/Pili_and_PaQa_counts_data/Pili_fluo/Mars/11/pilU'

strains=['mNG_pilH_fliC-', 'mNG_pilG_fliC-', 'pilB_mNG_fliC-', 'mNG_pilT_fliC-', 'mNG_pilU_fliC-']

for i in range(5):
    #combine_csv_files(root_path, data_path_old[i], strains[i], 'PolesData_'+strains[i]+'_old.csv', False)
    combine_csv_files(root_path, data_path[i], strains[i], 'PolesData_'+strains[i]+'.csv', False)
    combine_csv_files(root_path, root_path+strains[i]+'/', 'combined_data', 'PolesData_'+strains[i]+'_combined.csv', True)
    

C:\Users\tala\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
#set root folder
root_path = "C:/Users/lorenzo/Desktop/git/PhD_codes/Mechanosensation/Python_code/Pole_analysis/"
#set data folders
data_path="C:/Users/lorenzo/Desktop/mNG_pilG_pili"

strains=['mNG_pilG_fliC-']

for i in range(len(strains)):
    #combine_csv_files(root_path, data_path_old[i], strains[i], 'PolesData_'+strains[i]+'_old.csv', False)
    combine_csv_files(root_path, data_path, strains[i], 'PolesData_'+strains[i]+'.csv', False)
    combine_csv_files(root_path, root_path+strains[i]+'/', 'combined_data', 'PolesData_'+strains[i]+'_combined.csv', True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [4]:

df = pd.read_csv(saved_file, sep=',', na_values='*')
df['PoleRange']=df['AreaPoleDim']/df['CellArea']
pole_range=df.PoleRange.mean()
print(pole_range)
ref_ratio = (2*pole_range)/(1-2*pole_range)
df['PercentTotalFluoDim']=df['TotalFluorescencePoleDim']/df['CellTotalFluorescence']
df['PercentTotalFluoBright']=df['TotalFluorescencePoleBright']/df['CellTotalFluorescence']
df['PercentTotalFluoCenter']=(df['CellTotalFluorescence']-df['TotalFluorescencePoleBright']-df['TotalFluorescencePoleDim'])/df['CellTotalFluorescence']

df['polar_ratio']=(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright'])/(df['CellTotalFluorescence']-(df['TotalFluorescencePoleDim']+df['TotalFluorescencePoleBright']))/((2*df['PoleRange']/(1-2*df['PoleRange'])))

df['TotalPili']=df['Nb_Pili_PoleDim']+df['Nb_Pili_PoleBright']
df['TotalFlagella']=df['Nb_Flagella_PoleDim']+df['Nb_Flagella_PoleBright']




y_param1='Nb_Pili_PoleDim'
y_param2='Nb_Pili_PoleBright'
tot_pili_limit=15
Ncells_per_pili_Dim=np.empty((tot_pili_limit), dtype='int16')
Ncells_per_pili_Bright=np.empty((tot_pili_limit), dtype='int16')
N=len(df)
print('Dim pole:')
print('NCells='+str(N), end=', ')
for i in range(tot_pili_limit):
    Ncells_per_pili_Dim[i]=len(df.loc[(df[y_param1]==i)])
    if (i<tot_pili_limit-1):
        print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]), end=', ')
    else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]))
print('Bright pole:')
print('Ncells='+str(N), end=', ')
for i in range(tot_pili_limit):
    Ncells_per_pili_Bright[i]=len(df.loc[(df[y_param2]==i)])
    if (i<tot_pili_limit-1):
        print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]), end=', ')
    else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]))
        
df.head()

NameError: name 'saved_file' is not defined

In [10]:
sumPoles=(Ncells_per_pili_Dim+Ncells_per_pili_Bright)
Prob_Dim_if_nb_pili = [(Ncells_per_pili_Dim[i]/sumPoles[i]) for i in range(tot_pili_limit) if sumPoles[i]>0]
Prob_Bright_if_nb_pili=[(Ncells_per_pili_Bright[i]/sumPoles[i]) for i in range(tot_pili_limit) if sumPoles[i]>0]

Prob_per_pili_Dim= Ncells_per_pili_Dim/N
Prob_per_pili_Bright= Ncells_per_pili_Bright/N

In [11]:
p0 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='P(# pili)',
    title="Probability of having # pili"
)

x_1=range(tot_pili_limit)


p0.line(
    x=x_1,
    y=Prob_per_pili_Dim, 
    line_color = 'blue',
    #fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)


p0.line(
    x=x_1,
    y=Prob_per_pili_Bright, 
    line_color = 'red',
    #fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright pole'
)


bokeh.io.show(p0)

In [12]:
p1 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='P(# pili|pole state)',
    title="Probability of having # pili if bright or dim pole"
)

x_1=range(len(Prob_Dim_if_nb_pili))


p1.line(
    x=x_1,
    y=Prob_Dim_if_nb_pili, 
    line_color = 'blue',
    #fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)


p1.line(
    x=x_1,
    y=Prob_Bright_if_nb_pili, 
    line_color = 'red',
    #fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright pole'
)


bokeh.io.show(p1)

In [13]:
nb_pili=np.zeros((tot_pili_limit, 1))
boot_meanDim=np.zeros((tot_pili_limit, 1))
boot_medianDim=np.zeros((tot_pili_limit, 1))
boot_IC_meanDim=np.zeros((tot_pili_limit, 2))
boot_IC_medianDim=np.zeros((tot_pili_limit, 2))
boot_meanBright=np.zeros((tot_pili_limit, 1))
boot_medianBright=np.zeros((tot_pili_limit, 1))
boot_IC_meanBright=np.zeros((tot_pili_limit, 2))
boot_IC_medianBright=np.zeros((tot_pili_limit, 2))
for n_pili in range(tot_pili_limit):
    cell_fluorescence_arrayDim=df.TotalFluorescencePoleDim.loc[(df[y_param1]==n_pili)].values
    cell_fluorescence_arrayBright=df.TotalFluorescencePoleBright.loc[(df[y_param2]==n_pili)].values
    nb_pili[n_pili]=n_pili
    if (len(cell_fluorescence_arrayDim) > 0):
        bs_Dim_fluorescence_array=bootstrap_sampling(cell_fluorescence_arrayDim, len(cell_fluorescence_arrayDim), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_Dim_fluorescence_array)
        boot_meanDim[n_pili]=bs_means
        boot_IC_meanDim[n_pili,:]=IC_means
        boot_medianDim[n_pili]=bs_medians
        boot_IC_medianDim[n_pili,:]=IC_medians
    if (len(cell_fluorescence_arrayBright) > 0):
        bs_Bright_fluorescence_array=bootstrap_sampling(cell_fluorescence_arrayBright, len(cell_fluorescence_arrayBright), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_Bright_fluorescence_array)
        boot_meanBright[n_pili]=bs_means
        boot_IC_meanBright[n_pili,:]=IC_means
        boot_medianBright[n_pili]=bs_medians
        boot_IC_medianBright[n_pili,:]=IC_medians

names = ['TotalPili', 'bootMeanDim','bootMedianDim', 'bootMeanBright','bootMedianBright']
data = np.concatenate((nb_pili, boot_meanDim, boot_medianDim, boot_meanBright, boot_medianBright), axis=1)
df_boot_stats=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats['IC_meanDim']=list(boot_IC_meanDim)
df_boot_stats['IC_medianDim']=list(boot_IC_medianDim)
df_boot_stats['IC_meanBright']=list(boot_IC_meanBright)
df_boot_stats['IC_medianBright']=list(boot_IC_medianBright)
df_boot_stats

,TotalPili,bootMeanDim,bootMedianDim,bootMeanBright,bootMedianBright,IC_meanDim,IC_medianDim,IC_meanBright,IC_medianBright
0,0.0,26715.823875,24697.5,28525.452950,27286.0,"[22370.3515625, 32387.7734375]","[20802.0, 26942.687499999996]","[24566.3675, 32530.17875]","[24393.0, 32481.5]"
1,1.0,25707.138167,23631.5,28122.366750,25752.5,"[20416.744444444445, 32465.18472222222]","[19948.8625, 26905.0]","[23430.3875, 34028.08125]","[21330.0, 38131.0]"
2,2.0,30248.704333,29799.0,40283.895889,36651.0,"[26174.2625, 34343.5]","[25146.0, 36078.0]","[31539.875, 51496.47222222222]","[26432.0, 55330.0]"
3,3.0,25601.095000,25655.5,50619.984667,48190.0,"[20215.0, 31096.0]","[20215.0, 31096.0]","[26971.0, 77609.0]","[26971.0, 77609.0]"
4,4.0,0.000000,0.0,60755.000000,60755.0,"[0.0, 0.0]","[0.0, 0.0]","[60755.0, 60755.0]","[60755.0, 60755.0]"
5,5.0,0.000000,0.0,75010.000000,75010.0,"[0.0, 0.0]","[0.0, 0.0]","[75010.0, 75010.0]","[75010.0, 75010.0]"
6,6.0,0.000000,0.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
7,7.0,0.000000,0.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
8,8.0,0.000000,0.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
9,9.0,0.000000,0.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"


In [14]:
x_param1='TotalFluorescencePoleDim'#'PercentTotalFluoDim'
y_param1='Nb_Pili_PoleDim'
x_param2='TotalFluorescencePoleBright' #'PercentTotalFluoBright'
y_param2='Nb_Pili_PoleBright'
param2='TotalPili'
IC1='IC_medianDim'
IC2='IC_medianBright'
param1='bootMedianDim'
param3='bootMedianBright'
p2 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# Pili', 
    y_axis_type='linear',
    y_axis_label ='Pole Total fluorescence',
    title="Pole fluorescence vs #Pili (bootstrap median, 95% CI)"
)

p2.circle(
    source=df,
    x=y_param1,
    y=x_param1, 
    line_color = 'blue',
    fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)
p2.circle(
    source=df,
    x=y_param2,
    y=x_param2, 
    line_color = 'red',
    fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright Pole'
)

for n_pili in list(df_boot_stats[param2]):
    b = [n_pili,n_pili]
    ic=df_boot_stats.loc[(df_boot_stats[param2] == n_pili), [IC1]].values[0][0]
    p2.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )
    
for n_pili in list(df_boot_stats['TotalPili']):
    b = [n_pili,n_pili]
    ic=df_boot_stats.loc[(df_boot_stats[param2] == n_pili), [IC2]].values[0][0]
    p2.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )

p2.line(
    source = df_boot_stats.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'blue',
    alpha=0.6
)


p2.circle(
    source = df_boot_stats.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)

p2.line(
    source = df_boot_stats.loc[:, [param2, param3]],
    x = param2,
    y = param3,
    line_color = 'red',
    alpha=0.6
) 

p2.circle(
    source = df_boot_stats.loc[:, [param2, param3]],
    x = param2,
    y = param3,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)  

p2.output_backend = 'svg'

bokeh.io.show(p2)

In [15]:
x_param3='CellTotalFluorescence'
y_param3='polar_ratio'

print(x_param3+' vs '+y_param3+':')
[spearman_r, spearman_p]=stats.spearmanr(df[x_param3], df[y_param3])
print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))

[pearson_r, pearson_p]=stats.pearsonr(df[x_param3], df[y_param3])
print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))

CellTotalFluorescence vs polar_ratio:
Spearman correlation = 0.45304270318450696, p-value = 0.0025883699066680675
Pearson correlation = 0.6120513139480611, p-value = 1.6542358937078832e-05


In [16]:
x_param3='CellTotalFluorescence'
y_param3='polar_ratio'

p1 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='Total fluorescence', 
    y_axis_type='linear',
    y_axis_label ='Polar Ratio',
    title="Polar Ratio vs Cell Total fluorescence"
)

p1.circle(
    source=df,
    x=x_param3,
    y=y_param3, 
    line_color = 'black',
    fill_color = 'gray',
    alpha=0.7,
    #legend = labelsAll[i]
)

bokeh.io.show(p1)

In [17]:
y_param1='Nb_Pili_PoleDim'
y_param2='Nb_Pili_PoleBright'
x_param1='TotalFluorescencePoleDim'
x_param2='TotalFluorescencePoleBright'
parameterX='TotalPili'
parameterY='PoleTotalFluorescence'
poles_pili_dim=list(df[y_param1])
poles_pili_bright=list(df[y_param2])
poles_pili=poles_pili_dim+poles_pili_bright
poles_pili=np.asarray(poles_pili).reshape(len(poles_pili),1)
poles_fluo_dim=list(df[x_param1])
poles_fluo_bright=list(df[x_param2])
poles_fluo=poles_fluo_dim+poles_fluo_bright
poles_fluo=np.asarray(poles_fluo).reshape(len(poles_fluo),1)

names = [parameterX, parameterY]
data = np.concatenate((poles_pili, poles_fluo), axis=1)
df_long=pd.DataFrame(data=np.transpose(data), index=names).T

In [18]:
x_param3=parameterX
y_param3=parameterY

print(x_param3+' vs '+y_param3+':')
[spearman_r, spearman_p]=stats.spearmanr(df_long[x_param3], df_long[y_param3])
print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))

[pearson_r, pearson_p]=stats.pearsonr(df_long[x_param3], df_long[y_param3])
print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))

TotalPili vs PoleTotalFluorescence:
Spearman correlation = 0.2927979427515932, p-value = 0.0068750733771645225
Pearson correlation = 0.4434515310632639, p-value = 2.3928469862967177e-05


In [19]:
nb_pili=np.zeros((tot_pili_limit, 1))
boot_mean=np.zeros((tot_pili_limit, 1))
boot_median=np.zeros((tot_pili_limit, 1))
boot_IC_mean=np.zeros((tot_pili_limit, 2))
boot_IC_median=np.zeros((tot_pili_limit, 2))

for n_pili in range(tot_pili_limit):
    Pole_fluorescence_array=df_long.PoleTotalFluorescence.loc[(df_long[parameterX]==n_pili)].values
    nb_pili[n_pili]=n_pili
    if (len(Pole_fluorescence_array) > 0):
        bs_fluorescence_array=bootstrap_sampling(Pole_fluorescence_array, len(Pole_fluorescence_array), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_fluorescence_array)
        boot_mean[n_pili]=bs_means
        boot_IC_mean[n_pili,:]=IC_means
        boot_median[n_pili]=bs_medians
        boot_IC_median[n_pili,:]=IC_medians

names = ['TotalPili', 'bootMean','bootMedian']
data = np.concatenate((nb_pili, boot_mean, boot_median), axis=1)
df_boot_stats_tot=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats_tot['IC_mean']=list(boot_IC_mean)
df_boot_stats_tot['IC_median']=list(boot_IC_median)
df_boot_stats_tot

,TotalPili,bootMean,bootMedian,IC_mean,IC_median
0,0.0,27572.577028,26263.0,"[24621.236805555556, 30751.131944444445]","[23649.0, 27918.0]"
1,1.0,26570.487269,23631.5,"[22324.85096153846, 32020.360576923074]","[21321.0, 28574.0]"
2,2.0,36107.516267,32363.0,"[30505.548333333332, 43503.31333333334]","[28047.0, 38249.0]"
3,3.0,40997.232200,31096.0,"[25619.8, 60246.4]","[20215.0, 77609.0]"
4,4.0,60755.000000,60755.0,"[60755.0, 60755.0]","[60755.0, 60755.0]"
5,5.0,75010.000000,75010.0,"[75010.0, 75010.0]","[75010.0, 75010.0]"
6,6.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]"
7,7.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]"
8,8.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]"
9,9.0,0.000000,0.0,"[0.0, 0.0]","[0.0, 0.0]"


In [20]:
param2='TotalPili'
IC1='IC_median'
param1='bootMedian'

p3 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_range=[0, 100000],
    x_range=[-0.6, 8.6],
    y_axis_label ='pole fluorescence',
    title="Pole total fluorescence vs # pili (bootstrap median, 95% CI)"
)

p3.circle(
    source=df_long,
    x=parameterX,
    y=parameterY, 
    line_color = 'green',
    fill_color = 'white',
    alpha=0.4,
    #legend = labelsAll[i]
)

for n_pili in list(df_boot_stats_tot[param2]):
    b = [n_pili,n_pili]
    ic=df_boot_stats_tot.loc[(df_boot_stats_tot[param2] == n_pili), [IC1]].values[0][0]
    p3.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )

#p3.line(
 #   source = df_boot_stats_tot.loc[:, [param2, param1]],
  #  x = param2,
   # y = param1,
   # line_color = 'black',
  #  alpha=0.6
#)


p3.circle(
    source = df_boot_stats_tot.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)

p3.output_backend = 'svg'

bokeh.io.show(p3)

In [18]:
a=np.array(df.Nb_Pili_PoleDim)
b=np.array(df.Nb_Pili_PoleBright)
pili_nb_diff=(b-a)
c=np.zeros(len(a))
for i in range(len(a)):
    if(b[i] > a[i]):
        c[i]=1
    else:
        c[i]=0        
d=sum(c)
persentage_piliBright=d/len(a)
print(persentage_piliBright)
print(pili_nb_diff)

0.5
[ 5  0 -3  0  0  0  0 -2  3  0  0  0  1  2 -1  1  3  3  0  2  1  0 -1  6
  6  0 -2  0  2 -2  1  2  2  1 -2  1  5  1  5 -1]
